In [ ]:
%%HTML
<style type="text/css">
    table.dataframe td, table.dataframe th {
        border-style: solid;
    }
</style>

In [1]:
%matplotlib inline 
import pandas as pd 

In [2]:
import os
os.chdir('C://Users//2918261//Dropbox//Corner Office Interviews')
os.getcwd()
## C://Users//2918261//Courses//Oslo Summer School//notebooks//Interviews//Text 

'C:\\Users\\2918261\\Dropbox\\Corner Office Interviews'

In [3]:
pd.set_option('display.max_colwidth', 120)

In [4]:
interviews = pd.read_json('C:/Users/2918261/Dropbox/Corner Office Interviews/ComparisonCorpusText/Json Tables for ML/interviews_full.json')

## select men 

In [5]:
interviews = interviews[interviews['Female'] == 0]

## sort table alphabetically

In [6]:
interviews = interviews.sort_values('filename').copy()

In [7]:
interviews

,filename,text,Name,Gender,Organization,Current Age,NativeSpeaker,Traded,MBA,Sector_Grouped,Female,Sector_Grouped_Dic,Femininity,Masculinity
0,AaronBellCombined.txt,Very well. Thanks for having me.\nYeah. So a little bit about adroll. First of all we were founded in 2007 uh with a...,AaronBell,M,AdRoll,42,1,0,0,2,0,Communications,64.47,38.83
1,AaronLevieCombined.txt,Oh boy.Wow. Did we kick off this show\nThank you. I am glad that you re-professed your love for B2B. \nYeah its bee...,AaronLevie,M,Box Inc,33,1,1,0,3,0,Technology,63.35,37.73
3,AbeAnkumahCombined.txt,Being far away from home actually did not end up being as hard as people normally would have thought because I went...,AbeAnkumah,M,Nyansa,41,0,0,1,3,0,Technology,65.51,38.98
4,AdamNashCombined.txt,In 2012 I was seriously considering becoming a full time investor.\nI have always loved startups and venture capital...,AdamNash,M,Wealthfront,44,1,0,1,4,0,Other,62.76,40.87
7,AlanDabbiereCombined.txt,thank you I think somebody printed my wiki my wiki page my LinkedIn profile my Twitter account my I do not know wher...,AlanDabbiere,M,AirWatch LLC,57,1,0,1,3,0,Technology,63.52,36.30
8,AlanRMulallyCombined.txt,Glad to be here.\n Transformation would be my first word. And what I mean by that is that this is an American icon ...,AlanRMulally,M,Ford Motor Co Ltd,73,1,1,0,1,0,Consumer Discretionary,56.81,41.01
9,AlanTreflerCombined.txt,Prior to starting Pegasystems I worked for large New York banks as a systems integrator. Computers were getting fas...,AlanTrefler,M,Pegasystems Inc,63,1,1,0,3,0,Technology,58.71,48.18
10,AlastairMitchellCombined.txt,The most important thing you can do as a business leader is find the right team to help you run your company cofoun...,AlastairMitchell,M,Huddle,43,0,0,0,3,0,Technology,66.10,38.01
14,AmitSinghCombined.txt,Jonathan you are on fire I know I am not a Yankees fan of a Red Sox fan he just said that this payoff welcome to all...,AmitSingh,M,Google for Work,51,0,0,0,2,0,Communications,54.19,42.05
20,AndreDurandCombined.txt,That is a game changer let me be right up-front. A distributed ledger enables fairness without servers. The reason ...,AndreDurand,M,Ping Identity Corp,48,1,0,0,3,0,Technology,58.97,36.31


## topic modelling 

## import packages 

In [8]:
import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


In [9]:
data = interviews.text.values.tolist()

In [10]:
data = [re.sub("\'", "", sent) for sent in data]

In [11]:
pprint(data[:1])

['Very well. Thanks for having me.\n'
 'Yeah. So a little bit about adroll. First of all we were founded in 2007 uh '
 'with a mission to bring advertising to brands of all sizes back when we '
 'started. Uh the advertising world was very different. Um the advertisements '
 'you saw online work so different than the ones you saw offline you know a '
 'typical campaign would be nike running a sponsorship on the front page and '
 'there was not much space for all the emerging brands out there. Uh and so we '
 'set out to change that. And fast forward to today were now the most widely '
 'adopted um marketing tech platform out there for brands of all sizes. And we '
 'have two business units one focused on consumer called adroll and then a '
 'sister brand called inaudible which is focused on B to b advertisers. Uh we '
 'started the company in 2007. Um my background before adderall was almost all '
 'engineering.\n'
 'Um so I come from a very uh I love to build things. And when I began a

 'looking back on things and making sure that you are actually taking the time '
 'to learn from your failures. And I think by people understanding the value '
 'in that process then they are more likely to understand its okay to make '
 'mistakes.\n'
 'Um that is a good question actually. Uh one thing that you would have asked '
 'me about culture would be um well one thing I think its kind of there are '
 'two things I can relate to that I am. The first one is how do you measure '
 'culture which I think is really important. Um its sorta like how can it goes '
 'back to this idea I have where there is this. I think a myth out there were '
 'as a company gets bigger the culture gets worse it gets more bureaucratic '
 'against more stuffy things get slower there are diseconomies of scale. And I '
 'am hell-bent on destroying that myth. So actually as a company gets bigger '
 'how do you make sure that the company improves and I think I think '
 'measurements will be part of that. So we

## tokenizing and clean up with gensim simple_preprocess()

In [12]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['very', 'well', 'thanks', 'for', 'having', 'me', 'yeah', 'so', 'little', 'bit', 'about', 'adroll', 'first', 'of', 'all', 'we', 'were', 'founded', 'in', 'uh', 'with', 'mission', 'to', 'bring', 'advertising', 'to', 'brands', 'of', 'all', 'sizes', 'back', 'when', 'we', 'started', 'uh', 'the', 'advertising', 'world', 'was', 'very', 'different', 'um', 'the', 'advertisements', 'you', 'saw', 'online', 'work', 'so', 'different', 'than', 'the', 'ones', 'you', 'saw', 'offline', 'you', 'know', 'typical', 'campaign', 'would', 'be', 'nike', 'running', 'sponsorship', 'on', 'the', 'front', 'page', 'and', 'there', 'was', 'not', 'much', 'space', 'for', 'all', 'the', 'emerging', 'brands', 'out', 'there', 'uh', 'and', 'so', 'we', 'set', 'out', 'to', 'change', 'that', 'and', 'fast', 'forward', 'to', 'today', 'were', 'now', 'the', 'most', 'widely', 'adopted', 'um', 'marketing', 'tech', 'platform', 'out', 'there', 'for', 'brands', 'of', 'all', 'sizes', 'and', 'we', 'have', 'two', 'business', 'units', 'one

## lemmatizing 

In [13]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:2])

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Pool size changed, may indicate binary incompatibility. Expected 48 from C header, got 64 from PyObject
  return f(*args, **kwds)
C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: cymem.cymem.Address size changed, may indicate binary incompatibility. Expected 24 from C header, got 40 from PyObject
  return f(*args, **kwds)


['very well thank have so little bit adroll first be found mission bring advertising brand size back when start advertising world be very different advertisement see online work so different one see offline know typical campaign would be nike run sponsorship front page there be not much space all emerge brand out there so set change fast forward today be now most widely adopt market tech platform out there brand size have business unit focus consumer call adroll then sister brand call inaudible which be focus advertiser start company  background adderall be almost engineer so come very love build thing when begin ceo here can admit be pretty far  comfort zone have think way adapt  background know build code write program software how do build great organization where people thrive very quickly come believe people be everything get people can just get everything else wrong still be okay so start there have have employee company be number global office united state where san francisco ut

## creating document word matrix 

In [14]:
vectorizer = CountVectorizer(analyzer='word',
                             ngram_range = (1, 3),
                             min_df = 10,
                             #max_df = 0.9, # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

In [15]:
data_vectorized = vectorizer.fit_transform(data_lemmatized)

## sparsicity check 

Sparsicity is nothing but the percentage of non-zero datapoints in the document-word matrix, that is data_vectorized.
Since most cells in this matrix will be zero, I am interested in knowing what percentage of cells contain non-zero values.

In [16]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense > 0).sum()/data_dense.size)*100, "%")

Sparsicity:  10.996825153516717 %


## build LDA model

In [17]:
lda_model = LatentDirichletAllocation(n_topics=5,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=100,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:314: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=5, perp_tol=0.1,
             random_state=100, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


## check model performance with perplexity and log-likelyhood

In [18]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -3125109.4595918115
Perplexity:  1547.8309693779358
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 10,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'n_topics': 5,
 'perp_tol': 0.1,
 'random_state': 100,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


On a different note, perplexity might not be the best measure to evaluate topic models because it doesn’t consider the context and semantic associations between words. This can be captured using topic coherence measure

## looking for the best LDA model 

The most important tuning parameter for LDA models is n_components (number of topics), and learning_decay (which controls the learning rate).

Besides these, other possible search params could be learning_offset (downweigh early iterations. Should be > 1) and max_iter. These could be worth experimenting if you have enough computing resources.

Be warned, the grid search constructs multiple LDA models for all possible combinations of param values in the param_grid dict. So, this process can consume a lot of time and resources.

In [19]:
# Define Search Param
search_params = {'n_components': [10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9]}

# Init the Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, param_grid=search_params)

# Do the Grid Search
model.fit(data_vectorized)

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_components': [10, 15, 20, 25, 30], 'learning_decay': [0.5, 0.7, 0.9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

## selecting best topic model and parameters 

In [20]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

Best Model's Params:  {'learning_decay': 0.5, 'n_components': 10}
Best Log Likelihood Score:  -1089613.8424738233
Model Perplexity:  1567.09691931574


## model with best parameters 

In [27]:
lda_model = LatentDirichletAllocation(n_topics=None,               # Number of topics
                                      max_iter=10,               # Max learning iterations
                                      learning_method='batch',   
                                      random_state=None,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1, 
                                      learning_decay = 0.5,
                                      n_components = 4,   # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.5,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=4, n_jobs=-1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [28]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

C:\Users\2918261\AppData\Local\Continuum\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
3     -235.419525 -386.406525       1        1  41.446817
2     -644.242615 -243.458389       2        1  26.579305
1     -501.278473  165.359055       3        1  21.944715
0      -92.455345   22.410950       4        1  10.029162, topic_info=     Category         Freq            Term        Total  loglift  logprob
term                                                                     
1016  Default  1400.000000        customer  1400.000000  30.0000  30.0000
4532  Default   806.000000      technology   806.000000  29.0000  29.0000
753   Default  4103.000000         company  4103.000000  28.0000  28.0000
5005  Default  1479.000000             use  1479.000000  27.0000  27.0000
3013  Default  1509.000000             new  1509.000000  26.0000  26.0000
3554  Default  1030.000000         product  1030.000000  25.0000  25.0000
4218  Default   403.000000        software   403.000000  24.0000  24.0000
474   Default  2593.000000        business  2593.000000  23.0000  23.0000
2095  Default   625.000000        industry   625.000000  22.0000  22.0000
3902  Default   429.000000            risk   429.000000  21.0000  21.0000
670   Default   304.000000           cloud   304.000000  20.0000  20.0000
5252  Default  3969.000000            work  3969.000000  19.0000  19.0000
1050  Default   541.000000           datum   541.000000  18.0000  18.0000
2119  Default   400.000000      innovation   400.000000  17.0000  17.0000
598   Default  1203.000000          change  1203.000000  16.0000  16.0000
1325  Default   869.000000        employee   869.000000  15.0000  15.0000
448   Default  1204.000000           build  1204.000000  14.0000  14.0000
3195  Default  7512.000000          people  7512.000000  13.0000  13.0000
1180  Default   274.000000         digital   274.000000  12.0000  12.0000
4101  Default   524.000000         service   524.000000  11.0000  11.0000
3583  Default   361.000000         project   361.000000  10.0000  10.0000
5364  Default  2703.000000            year  2703.000000   9.0000   9.0000
2339  Default  4781.000000            know  4781.000000   8.0000   8.0000
5157  Default  2772.000000             way  2772.000000   7.0000   7.0000
2971  Default  2032.000000            need  2032.000000   6.0000   6.0000
2787  Default   655.000000          market   655.000000   5.0000   5.0000
2697  Default  3985.000000            make  3985.000000   4.0000   4.0000
1134  Default   350.000000          design   350.000000   3.0000   3.0000
967   Default  1041.000000          create  1041.000000   2.0000   2.0000
4503  Default  1440.000000            team  1440.000000   1.0000   1.0000
...       ...          ...             ...          ...      ...      ...
2107   Topic4    62.696173  infrastructure   118.176764   1.6658  -6.5230
1283   Topic4    64.295419         economy   125.680881   1.6294  -6.4979
2095   Topic4   191.108924        industry   625.989155   1.1132  -5.4085
1777   Topic4    83.767448      government   186.732916   1.4980  -6.2333
2910   Topic4   112.488908           model   318.200746   1.2598  -5.9385
3013   Topic4   296.111082             new  1509.152746   0.6711  -4.9706
5005   Topic4   291.800497             use  1479.579710   0.6762  -4.9853
598    Topic4   228.315941          change  1203.063090   0.6378  -5.2306
1050   Topic4   141.271790           datum   541.972461   0.9551  -5.7107
448    Topic4   206.026823           build  1204.172383   0.5341  -5.3333
1016   Topic4   217.337295        customer  1400.852953   0.4363  -5.2799
5252   Topic4   367.573798            work  3969.239456  -0.0797  -4.7544
5157   Topic4   298.596996             way  2772.581140   0.0712  -4.9622
4101   Topic4   130.183417         service   524.451293   0.9063  -5.7924
2697   Topic4   343.058574            make  3985.152719  -0.1528  -4.8234
474    Topic4   278.854583        business  2593.397855  

In [29]:
pyLDAvis.save_html(panel, 'combined_men4_lda.html')